In [131]:
# Imports

import numpy as np
import glob

from scipy.io import wavfile
from scipy.fft import fft
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

FS_RATE = 22050
SIGNAL_LEN = 66150

In [122]:
def read_wav_file(file_name):
    return wavfile.read(file_name)[1]

In [123]:
# Read files and labels
signals = []
labels = []

with open('./data/labels.csv') as labels_file:
    labels_raw = dict([line.strip().split(',') for line in labels_file.readlines()[1:]])

    for file_name in glob.glob('./data/audio/*.wav'):
        signals.append(read_wav_file(file_name))
        labels.append(labels_raw[file_name.split('\\')[1]])

In [124]:
# Convert to np array
data = np.array(signals)
labels = np.array(labels)

In [125]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)